# Schema Proposal (Structured)

The next phase is to propose a schema for the knowledge graph, based on the established user goal and approved import files.

There are two main options:
1. Structured data (e.g. CSV, JSON, XML)
2. Unstructured data (e.g. PDF, HTML, Markdown, Plain Text)

First, we'll tackle the structured data case, which is relatively straightforward yet has subtle modeling challenges.

## Agent

- An agent that suggests a schema for the knowledge graph, based on the established user goal.
- Input: `approved_user_goal`, `approved_files`
- Output: `approved_schema`, a dictionary containing the schema for the knowledge graph.
- Tools: `get_approved_user_goal`, `get_approved_files`, `sample_file`, 
        `propose_node_construction`, `propose_relationship_construction`, `approve_proposed_construction_plan`

## Workflow

1. The context is initialized with an `approved_user_goal` and `approved_files`
2. For each file, determine whether it represents a node or a relationship.
3. For each node file, propose a node construction (file --> label, properties).
4. For each relationship file, propose a relationship construction (file --> source and target nodes, relationship type and properties).
5. Present the construction proposal and ask for approval.
6. The user approves the construction proposal.
7. The construction proposal is saved in the context state as `approve_proposed_construction_plan`.


## Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [ ]:
# Import necessary libraries
import os
from pathlib import Path

from itertools import islice

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types # For creating message Content/Parts

# For type hints
from typing import Dict, Any, List

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nEnvironment configured.")

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

## Define the Structured Schema Proposal Agent

### Agent Instructions

In [ ]:
# First, define the instruction to describe what the agent should do
structured_schema_proposal_agent_instruction = """
        You are an expert at knowledge graph modeling with property graphs. Propose an appropriate
        schema based on the user goal and list of approved files.
        
        Prepare for the task:
        - get the user goal using the 'get_approved_user_goal' tool
        - get the list of approved files using the 'get_approved_files' tool

        Think carefully and collaborate with the user:
        1. For each approved file, consider whether it represents a node or relationship. If you're unsure, use the 'sample_file' tool to get a better understanding of the file contents.
        2. For a node file, propose a node construction using the 'propose_node_construction' tool
        3. For a relationship file, propose a relationship construction using the 'propose_relationship_construction' tool
        4. After proposing a construction for each file, present the proposed schema to the user, asking for their approval
        5. If they disapprove, consider their feedback and go back to step 1
        6. If the approve, use the 'approve_schema' tool to record the approval
        """


### Tool Definitions

In [ ]:
# import tools defined in previous notebook
from tools import get_approved_user_goal, get_approved_files, sample_file


In [ ]:
#  Tool: Propose Node Construction

PROPOSED_CONSTRUCTION_PLAN = "proposed_construction_plan"
NODE_CONSTRUCTION = "node_construction"

def propose_node_construction(approved_file: str, proposed_label: str, unique_column_name: str, proposed_properties: list[str], tool_context:ToolContext) -> dict:
    f"""Propose a node construction for an approved file that supports the user goal.

    The construction plan will be saved to {PROPOSED_CONSTRUCTION_PLAN} list of dictionaries.
    Each dictionary will have the following keys:
    - construction_type: "node"
    - source_file: The approved file to propose a node construction for
    - label: The label of the node
    - unique_column_name: The name of the column that will be used to uniquely identify constructed nodes
    - properties: A list of properties for the node

    Args:
        approved_file: The approved file to propose a node construction for
        proposed_label: The proposed label for constructed nodes
        unique_column_name: The name of the column that will be used to uniquely identify constructed nodes
        tool_context: The tool context

    Returns:
        dict: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {NODE_CONSTRUCTION} key with the construction plan for the node
                If 'error', includes an 'error_message' key.
                The 'error_message' may have instructions about how to handle the error.
    """
    construction_plan = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, [])
    node_construction_rule = {
        "construction_type": "node",
        "source_file": approved_file,
        "label": proposed_label,
        "unique_column_name": unique_column_name,
        "properties": proposed_properties
    }   
    construction_plan.append(node_construction_rule)
    tool_context.state[PROPOSED_CONSTRUCTION_PLAN] = construction_plan
    return tool_success(NODE_CONSTRUCTION, node_construction_rule)


In [ ]:
#  Tool: Propose Relationship Construction

RELATIONSHIP_CONSTRUCTION = "relationship_construction"

def propose_relationship_construction(approved_file: str, proposed_relationship_type: str, from_node_column: str, to_node_column: str, proposed_properties: list[str], tool_context:ToolContext) -> dict:
    f"""Propose a relationship construction for an approved file that supports the user goal.

    The construction plan will be saved to {PROPOSED_CONSTRUCTION_PLAN} list of dictionaries.
    For relationships, the dictionary will have the following keys:
    - construction_type: "relationship"
    - source_file: The approved file to propose a node construction for
    - proposed_relationship_type: The type of the relationship
    - from_node_column: The name of the column that will be used to uniquely identify constructed nodes
    - to_node_column: The name of the column that will be used to uniquely identify constructed nodes
    - proposed_properties: A list of properties for the node

    Args:
        approved_file: The approved file to propose a node construction for
        proposed_label: The proposed label for constructed nodes
        unique_column_name: The name of the column that will be used to uniquely identify constructed nodes
        tool_context: The tool context

    Returns:
        dict: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {RELATIONSHIP_CONSTRUCTION} key with the construction plan for the node
                If 'error', includes an 'error_message' key.
                The 'error_message' may have instructions about how to handle the error.
    """
    construction_plan = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, [])
    relationship_construction_rule = {
        "construction_type": "relationship",
        "source_file": approved_file,
        "proposed_relationship_type": proposed_relationship_type,
        "from_node_column": from_node_column,
        "to_node_column": to_node_column,
        "proposed_properties": proposed_properties
    }   
    construction_plan.append(relationship_construction_rule)
    tool_context.state[PROPOSED_CONSTRUCTION_PLAN] = construction_plan
    return tool_success(RELATIONSHIP_CONSTRUCTION, relationship_construction_rule)


In [ ]:
APPROVED_CONSTRUCTION_PLAN = "approved_construction_plan"

# Tool: Approve the proposed construction plan
def approve_proposed_construction_plan(tool_context:ToolContext) -> dict:
    """Approve the proposed construction plan."""
    tool_context.state[APPROVED_CONSTRUCTION_PLAN] = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, [])
    return tool_success(APPROVED_CONSTRUCTION_PLAN, tool_context.state[APPROVED_CONSTRUCTION_PLAN])
    

In [ ]:
# Tool: Get Proposed construction Plan

def get_proposed_construction_plan(tool_context:ToolContext) -> dict:
    """Get the proposed construction plan."""
    return tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, [])

In [ ]:
# List of tools for the structured schema proposal agent
structured_schema_proposal_agent_tools = [get_approved_user_goal, get_approved_files, propose_node_construction, propose_relationship_construction, get_proposed_construction_plan, approve_proposed_construction_plan ]

### Construct the Agent

In [ ]:
# Finally, construct the agent

structured_schema_proposal_agent = Agent(
    name="structured_schema_proposal_agent_v1",
    model=llm, # defined earlier in a variable
    description="Proposes a knowledge graph schema based on the user goal and approved file list.",
    instruction=structured_schema_proposal_agent_instruction,
    tools=structured_schema_proposal_agent_tools,
)

print(f"Agent '{structured_schema_proposal_agent.name}' created.")

---

## Interact with the Agent



In [ ]:
# Define an Agent Caller Utility
# This will provide a simple "call" interface and access to the session

from helpers import make_agent_caller

structured_schema_proposal_caller = make_agent_caller(structured_schema_proposal_agent, {
    "approved_user_goal": {
        "kind_of_graph": "movie graph", # TODO: change to a BOM graph
        "description": "Movies, actors and acted-in relationships for study of co-acting group behaviors."
    },
    "approved_files": ['acting_roles.csv', 'actors.csv', 'movies.csv']
})


In [ ]:
# Run the Initial Conversation
await structured_schema_proposal_caller.call("How can these files be imported?", True)

print("Proposed construction plan: ", structured_schema_proposal_caller.session.state[PROPOSED_CONSTRUCTION_PLAN])


In [ ]:
# Agree with the file suggestions
await structured_schema_proposal_caller.call("Yes, let's do it!", True)

print("Approved construction plan: ", structured_schema_proposal_caller.session.state[APPROVED_CONSTRUCTION_PLAN])



---

Congratulations\! You've created a basic human-in-the-loop interaction, with a structured result.


---
## Bonus, An Interactive Conversation

Now, let's make this interactive so you can ask your own questions! Run the cell below. It will prompt you to enter your queries directly.

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await file_suggestion_caller.call(user_query, True)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()